In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from joblib import dump, load



# For now filter the songs data to only the ones in playlists to save on memory

In [2]:
playlists_tracks = pd.read_csv("LastFM-Playlist-Tracks.csv")
tracks = pd.read_csv("LastFM-Tracks.csv")
songs_filtered = pd.read_csv("songs-filtered-machine-learning.csv")

In [3]:
songs_filtered

,Unnamed: 0,TrackID,TrackName,Popularity,Duration,Explicit,Artists,ArtistsID,ReleaseYear,Danceability,...,Liveness,Valence,Tempo,TimeSignature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists,popularity_red
0,9815,7s8v7hQ80FZyL1N9FHZW5W,Пока боги спят,30,426000,0,['Сергей Маврин'],['77PePiWlGlG80CJEgPah3B'],2004,0.260,...,0.2400,0.457,119.535,3,['Сергей Маврин'],[],['Сергей Маврин'],Сергей Маврин+Пока боги спят,"['russian_heavy_metal', 'russian_metal']",6
1,11564,38Mxx28rpORDuDAGuZe1F4,Ben Seni Sevduğumi,33,172500,0,['Şevval Sam'],['5zkh6MICzIQIUoA9NfZpBx'],2008,0.487,...,0.2200,0.572,78.869,5,['Şevval Sam'],[],['Şevval Sam'],Şevval Sam+Ben Seni Sevduğumi,"['karadeniz_turkuleri', 'turkish_pop', 'turkis...",6
2,11704,6mGP1LAOLKTUcu1TpAF8vy,Lovelock,38,240425,0,['Łąki Łan'],['2fyLjdAfNDXGANys1KoAzo'],2012,0.705,...,0.2410,0.851,139.988,4,['Łąki Łan'],[],['Łąki Łan'],Łąki Łan+Lovelock,['polish_alternative'],7
3,11706,4G1GhiYTkfgsmMv1xlL6jO,"Zabij się, zabij",26,236773,0,['Łzy'],['49kDJkXVeVB2KjvJmAwq0f'],1998,0.568,...,0.1130,0.616,120.858,4,['Łzy'],[],['Łzy'],"Łzy+Zabij się, zabij",['polish_pop'],5
4,11711,10hWSYRQZ4B1IjPafMhiKr,Jestem jaka jestem,27,230574,0,['Łzy'],['49kDJkXVeVB2KjvJmAwq0f'],2002,0.523,...,0.3680,0.558,93.020,4,['Łzy'],[],['Łzy'],Łzy+Jestem jaka jestem,['polish_pop'],5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10042,523358,0tNoPlTA4WasOiJNyRbRvh,Like a Surgeon,17,210040,0,"['""Weird Al"" Yankovic']",['1bDWGdIC2hardyt55nlQgG'],1988,0.837,...,0.0506,0.962,126.136,4,"['""Weird Al"" Yankovic']",['Weird Al'],"['""Weird Al"" Yankovic']","""Weird Al"" Yankovic+Like a Surgeon","['comic', 'parody', 'comedy_rock']",3
10043,523363,022XE9RZVU3a9nULENXfnm,I Lost On Jeopardy,36,208933,0,"['""Weird Al"" Yankovic']",['1bDWGdIC2hardyt55nlQgG'],1984,0.887,...,0.0507,0.836,113.262,4,"['""Weird Al"" Yankovic']",['Weird Al'],"['""Weird Al"" Yankovic']","""Weird Al"" Yankovic+I Lost On Jeopardy","['comic', 'parody', 'comedy_rock']",7
10044,523368,073V7jY2AD5JXEnej4JUd1,Eat It,21,200440,0,"['""Weird Al"" Yankovic']",['1bDWGdIC2hardyt55nlQgG'],1988,0.763,...,0.0701,0.912,147.434,4,"['""Weird Al"" Yankovic']",['Weird Al'],"['""Weird Al"" Yankovic']","""Weird Al"" Yankovic+Eat It","['comic', 'parody', 'comedy_rock']",4
10045,523445,0VVMYrDOTj2MQSXID2WMTI,Tell Everybody I Know,41,189093,0,"[""Keb' Mo'""]",['6iDaoPZVgxrTkndDCisX8F'],1994,0.657,...,0.0501,0.945,166.427,4,[' Mo'],"[""Keb' Mo'""]",[' Mo'],Mo+Tell Everybody I Know,[],8


In [4]:
songs_filtered.drop(columns='Unnamed: 0', inplace=True)

In [5]:
songs_filtered['consolidates_genre_lists'] = songs_filtered['consolidates_genre_lists'] .apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [6]:
# tfidf can't handle nulls so fill any null values with an empty list
songs_filtered['consolidates_genre_lists'] = songs_filtered['consolidates_genre_lists'].apply(lambda d: d if isinstance(d, list) else [])

In [7]:
float_cols = songs_filtered.dtypes[(songs_filtered.dtypes == 'float64') | (songs_filtered.dtypes == 'int64')].index.values

In [8]:
songs_filtered['artists_upd_v1'] = songs_filtered['Artists'].apply(lambda x: re.findall(r"'([^']*)'", x))
songs_filtered['artists_upd_v2'] = songs_filtered['Artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
songs_filtered['artists_upd'] = np.where(songs_filtered['artists_upd_v1'].apply(lambda x: not x), songs_filtered['artists_upd_v2'], songs_filtered['artists_upd_v1'] )

In [9]:
songs_filtered["len"] = songs_filtered['artists_upd'].apply(lambda x: len(x))

In [10]:
songs_filtered_sorted = songs_filtered.sort_values(by='len', ascending=False)
songs_filtered_sorted

,TrackID,TrackName,Popularity,Duration,Explicit,Artists,ArtistsID,ReleaseYear,Danceability,Energy,...,Valence,Tempo,TimeSignature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists,popularity_red,len
7549,0kNwj2qeErviyMF4lrhgn9,The Hawk Talks,0,197267,0,"['Duke Ellington', 'Clark Terry', 'Willie Cook...","['4F7Q5NV6h5TSwCainz8S5A', '62xdwGEOLoGQxKUqoJ...",1954,0.4500,0.6210,...,0.4270,110.788,4,"[Duke Ellington, Clark Terry, Willie Cook, Cat...",[],"[Duke Ellington, Clark Terry, Willie Cook, Cat...",Duke Ellington+The Hawk Talks,"[harlem_renaissance, jazz_piano, soul_jazz, ha...",0,15
244,1IEqRtID6cqzspmededRM1,Sinfonia Concertante in E-Flat Major: II. Adagio,11,461133,0,"['Wolfgang Amadeus Mozart', 'Alfred Tolksdorf'...","['4NJhFmfw43RLBLjQvxDuRS', '1PVQx4ZEtmh2KO5nHR...",1967,0.0663,0.0245,...,0.0399,69.117,4,"[Wolfgang Amadeus Mozart, Alfred Tolksdorf, Ka...",[],"[Wolfgang Amadeus Mozart, Alfred Tolksdorf, Ka...",Wolfgang Amadeus Mozart+Sinfonia Concertante i...,"[orchestra, classical_performance, classical, ...",2,7
7665,7tSGcD2FFsf2VaWWIaqcfN,Utopia and Visions,34,393400,0,"['Don Cherry', 'Tommy Koverhult', 'Tommy Goldm...","['3uPWecBPNXAChysw1uOJwI', '6G3CmG2kj7vybt9ALh...",1972,0.2690,0.3860,...,0.5980,79.661,4,"[Don Cherry, Tommy Koverhult, Tommy Goldman, M...",[],"[Don Cherry, Tommy Koverhult, Tommy Goldman, M...",Don Cherry+Utopia and Visions,"[jazz_trumpet, turkish_modern_jazz, cool_jazz,...",6,6
5768,3AKAAbFYypDMas1MV8Nhot,"Bach: Brandenburg Concerto No. 2 in F Major, B...",36,307200,0,"['Johann Sebastian Bach', 'Mark Bennett', 'Rac...","['5aIqB5nVVvmFsvSdExz408', '0EWX23pjbQ3h28fGpV...",1989,0.6530,0.2910,...,0.8280,96.709,4,"[Johann Sebastian Bach, Mark Bennett, Rachel B...",[],"[Johann Sebastian Bach, Mark Bennett, Rachel B...",Johann Sebastian Bach+Bach: Brandenburg Concer...,"[baroque_violin, orchestra, classical_performa...",7,6
7772,6Wt5QBzHp9vdSLAFlvdnDQ,"I Need a Girl (Pt. 2) [feat. Loon, Ginuwine & ...",69,285587,0,"['Diddy', 'Ginuwine', 'Loon', 'Mario Winans', ...","['59wfkuBoNyhDMQGCljbUbA', '7r8RF1tN2A4CiGEplk...",2004,0.7130,0.4710,...,0.7810,199.765,4,"[Diddy, Ginuwine, Loon, Mario Winans, Tammy Ru...",[],"[Diddy, Ginuwine, Loon, Mario Winans, Tammy Ru...","Diddy+I Need a Girl (Pt. 2) [feat. Loon, Ginuw...","[southern_hip_hop, urban_contemporary, dance_p...",13,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422,4udWxUu33DhKGP7uhSbUiN,Reza vela,55,296640,0,['O Rappa'],['1A5QJAC1vdhbhPE25Q0x0f'],2003,0.6360,0.8860,...,0.6710,87.268,4,[O Rappa],[],[O Rappa],O Rappa+Reza vela,"[rock_nacional_brasileiro, brazilian_rock, bra...",11,1
3423,1HtiK1p8rDPE4z57dlYTvM,O salto,40,377507,0,['O Rappa'],['1A5QJAC1vdhbhPE25Q0x0f'],2003,0.5280,0.9650,...,0.7570,157.003,4,[O Rappa],[],[O Rappa],O Rappa+O salto,"[rock_nacional_brasileiro, brazilian_rock, bra...",8,1
3424,1A47ISHc08cBeIA3hifQXn,O que sobrou do céu,55,232667,0,['O Rappa'],['1A5QJAC1vdhbhPE25Q0x0f'],1999,0.3740,0.9460,...,0.7020,105.731,4,[O Rappa],[],[O Rappa],O Rappa+O que sobrou do céu,"[rock_nacional_brasileiro, brazilian_rock, bra...",11,1
3425,3cQmBe2ikbBn0K7IYDl2sp,Monstro invisível,53,280547,0,['O Rappa'],['1A5QJAC1vdhbhPE25Q0x0f'],2008,0.5530,0.8500,...,0.7160,172.919,4,[O Rappa],[],[O Rappa],O Rappa+Monstro invisível,"[rock_nacional_brasileiro, brazilian_rock, bra...",10,1


In [11]:
playlist_songs_with_ids = playlists_tracks.merge(tracks, how='left', left_on='track', right_on='trackid')
playlist_songs_with_ids

,playlist,track,trackid,spotifyid,title,artist,album,imgurl,imgfile,popularity,duration
0,1,1,1,02JnoHSIDbpVW40uipjKcL,Roxanne,1,1,NaN,NaN,0.00,192.948
1,1,2,2,01HNAQL86oZsKECUfJiAwk,22 Acacia Avenue,2,2,NaN,NaN,0.00,395.720
2,1,3,3,0eCxvvcJUgMmFOEv0tphgh,Lovely Ladies,3,3,NaN,NaN,0.00,231.946
3,1,4,4,1pT9RHD2v3aHqENfVaFPw4,Sweet Painted Lady,4,4,NaN,NaN,0.32,234.733
4,1,5,5,68PVZq98OxgeWBWbskYQLt,"Build God, Then We'll Talk",5,5,NaN,NaN,0.36,220.720
...,...,...,...,...,...,...,...,...,...,...,...
1024706,10697,416541,416541,2dkw6BVxhDSBr91gT9hOGd,Stammgast,14577,187057,NaN,NaN,0.16,338.314
1024707,10697,410904,410904,0BkKep2Tf9JlPr2g1zbJiw,Treibsand,31649,174485,NaN,NaN,0.08,416.106
1024708,10697,352239,352239,7i5ZRYvZjg0YHFI5P1qfsb,Elevator,11893,175485,NaN,NaN,0.27,297.465
1024709,10697,410905,410905,0mYrkZjdEbCWfTSLAHU4fO,Well Done,44551,199938,NaN,NaN,0.00,501.453


In [12]:
#function to build entire feature set
def create_feature_set(df, float_cols):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)

    # explicity_ohe = ohe_prep(df, 'Explicit','exp')    
   
    # popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns)

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled], axis = 1)
     
    #add song id
    final['TrackID']=df['TrackID'].values
    
    return final

In [13]:
complete_feature_set = create_feature_set(songs_filtered, float_cols=float_cols)

c:\Users\edsta\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
complete_feature_set

,genre|_hip_hop,genre|a_cappella,genre|abstract,genre|abstract_hip_hop,genre|accordion,genre|acid_house,genre|acid_jazz,genre|acid_rock,genre|acid_trance,genre|acousmatic,...,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,TimeSignature,popularity_red,TrackID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.071320,0.022892,0.000265,0.241206,0.460685,0.543605,0.6,0.3750,7s8v7hQ80FZyL1N9FHZW5W
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.041017,0.968876,0.000000,0.221106,0.576613,0.358670,1.0,0.3750,38Mxx28rpORDuDAGuZe1F4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.125541,0.037249,0.000006,0.242211,0.857863,0.636619,0.8,0.4375,6mGP1LAOLKTUcu1TpAF8vy
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.030087,0.012450,0.000000,0.113568,0.620968,0.549622,0.8,0.3125,4G1GhiYTkfgsmMv1xlL6jO
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.035065,0.023695,0.000000,0.369849,0.562500,0.423024,0.8,0.3125,10hWSYRQZ4B1IjPafMhiKr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.038528,0.347390,0.000000,0.050854,0.969758,0.573624,0.8,0.1875,0tNoPlTA4WasOiJNyRbRvh
10043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.053788,0.348394,0.000009,0.050955,0.842742,0.515078,0.8,0.4375,022XE9RZVU3a9nULENXfnm
10044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.074892,0.117470,0.000000,0.070452,0.919355,0.670481,0.8,0.2500,073V7jY2AD5JXEnej4JUd1
10045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.067424,0.416667,0.000251,0.050352,0.952621,0.756854,0.8,0.5000,0VVMYrDOTj2MQSXID2WMTI


In [25]:
#filter to only playlist 
playlist_1065 = playlist_songs_with_ids[playlist_songs_with_ids['playlist']==1065]
playlist_1065

,playlist,track,trackid,spotifyid,title,artist,album,imgurl,imgfile,popularity,duration
65215,1065,21856,21856,5sxIzTn9gJlsIWoMNkKyag,2112,3991,15774,NaN,NaN,0.00,1233.586
65216,1065,48092,48092,20bJTQOaoHqeCHBhqIgcTl,The Ultra-Violence,11126,33038,NaN,NaN,0.42,633.600
65217,1065,42969,42969,0b47En2lJ0yP6ZKhyOG9Kd,Closer To The Heart,3991,29755,NaN,NaN,0.20,171.866
65218,1065,48093,48093,0wu0hJZ61dT5uSzL0tuPnQ,Canada,14346,33039,NaN,NaN,0.12,331.808
65219,1065,48094,48094,0vqSdwyJMafqz53TxsqXOD,Thicker Than Blood,11126,24283,NaN,NaN,0.07,158.333
65220,1065,42957,42957,0vwEcik5vpKs1tgwzKltBw,La Villa Strangiato,3991,7166,NaN,NaN,0.18,575.066
65221,1065,12726,12726,20ww15KemlpsXO1KdiOigX,Mordecai,5024,9125,NaN,NaN,0.34,347.760
65222,1065,48095,48095,0ff6FUCAt7YsWgS7wih7Pp,End Of The Beginning,14347,33040,NaN,NaN,0.34,706.400
65223,1065,48096,48096,1LGklBuwUCv4pTeD7CIq0b,Hellion,4957,33041,NaN,NaN,0.00,236.893
65224,1065,20942,20942,1E0ONfqq74b4gYhdlyhMSB,YYZ,3991,15334,NaN,NaN,0.48,266.066


In [26]:
def generate_playlist_feature(complete_feature_set, playlist_df):
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['TrackID'].isin(playlist_df['spotifyid'].values)]
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['spotifyid']], how = 'inner', left_on = 'TrackID', right_on='spotifyid')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['TrackID'].isin(playlist_df['spotifyid'].values)]
    complete_feature_set_playlist.drop(columns=['TrackID','spotifyid'],inplace=True)

    return complete_feature_set_playlist.sum(axis = 0), complete_feature_set_nonplaylist

In [27]:
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlist_1065)

In [18]:
complete_feature_set_nonplaylist.shape

(10045, 2041)

# Generate Playlist Recommendations

In [19]:
def generate_playlist_recos(df, features, nonplaylist_features):

    non_playlist_df = df[df['TrackID'].isin(nonplaylist_features['TrackID'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('TrackID', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(10)
    return non_playlist_df_top_40


In [29]:
top_40_user_1065 = generate_playlist_recos(songs_filtered, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)


C:\Users\edsta\AppData\Local\Temp\ipykernel_17012\3211393621.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Here are the recommendations

In [30]:
top_40_user_1065

,TrackID,TrackName,Popularity,Duration,Explicit,Artists,ArtistsID,ReleaseYear,Danceability,Energy,...,Tempo,TimeSignature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists,popularity_red,len,sim
4086,2R6UrJ8uWbSIiHWmvRQvN8,Whiskey In The Jar,73,304693,0,['Metallica'],['2ye2Wgw4gimLv2eAKyk1NB'],1998,0.511,0.970,...,132.986,4,[Metallica],[],[Metallica],Metallica+Whiskey In The Jar,"[old_school_thrash, metal, rock, hard_rock, th...",14,1,0.955792
4098,0zZPCkSMAECtxBQHWJ0enW,Blitzkrieg,41,216960,0,['Metallica'],['2ye2Wgw4gimLv2eAKyk1NB'],1998,0.493,0.970,...,98.736,4,[Metallica],[],[Metallica],Metallica+Blitzkrieg,"[old_school_thrash, metal, rock, hard_rock, th...",8,1,0.953566
4090,1R29PfqelwVMRPhg5fn0lD,Slither,42,313200,0,['Metallica'],['2ye2Wgw4gimLv2eAKyk1NB'],1997,0.509,0.851,...,110.364,4,[Metallica],[],[Metallica],Metallica+Slither,"[old_school_thrash, metal, rock, hard_rock, th...",8,1,0.952824
4096,0rDPQzWWmLTv9deHn4ulG7,Hero Of The Day,54,261907,0,['Metallica'],['2ye2Wgw4gimLv2eAKyk1NB'],1996,0.350,0.902,...,115.909,4,[Metallica],[],[Metallica],Metallica+Hero Of The Day,"[old_school_thrash, metal, rock, hard_rock, th...",10,1,0.948256
4094,0R9GrbRcpBRdwDlVPj2wEQ,My Friend Of Misery,55,407773,0,['Metallica'],['2ye2Wgw4gimLv2eAKyk1NB'],1991,0.582,0.819,...,119.513,4,[Metallica],[],[Metallica],Metallica+My Friend Of Misery,"[old_school_thrash, metal, rock, hard_rock, th...",11,1,0.947081
4100,1Bpa3yoBAqpcc8mRC9jacn,Astronomy,47,397840,0,['Metallica'],['2ye2Wgw4gimLv2eAKyk1NB'],1998,0.345,0.778,...,86.221,4,[Metallica],[],[Metallica],Metallica+Astronomy,"[old_school_thrash, metal, rock, hard_rock, th...",9,1,0.946627
4099,0cKWKhciVCYNyxFVnV1Y4R,Bad Seed,42,245333,0,['Metallica'],['2ye2Wgw4gimLv2eAKyk1NB'],1997,0.532,0.952,...,126.917,4,[Metallica],[],[Metallica],Metallica+Bad Seed,"[old_school_thrash, metal, rock, hard_rock, th...",8,1,0.945376
7919,7ag4qTNensFZLQP4raCIs6,Voracious Souls,32,339000,0,['Death Angel'],['6KVc8Llznru8n9LVCYe9dz'],1987,0.192,0.988,...,162.068,4,[Death Angel],[],[Death Angel],Death Angel+Voracious Souls,"[old_school_thrash, metal, hard_rock, thrash_m...",6,1,0.943264
4088,1bEAFB39TyDfY3KtDtK7aI,The More I See,40,288667,0,['Metallica'],['2ye2Wgw4gimLv2eAKyk1NB'],1998,0.304,0.957,...,94.840,4,[Metallica],[],[Metallica],Metallica+The More I See,"[old_school_thrash, metal, rock, hard_rock, th...",8,1,0.934355
2064,0Auq7v8eMbFXMi6ZaMCb09,Kill Again,42,296040,0,['Slayer'],['1IQ2e1buppatiN1bxUVkrk'],1985,0.301,0.930,...,99.363,4,[Slayer],[],[Slayer],Slayer+Kill Again,"[speed_metal, old_school_thrash, metal, hard_r...",8,1,0.926429


# Make recommendations for playlist 100

In [46]:
playlist_100 = playlist_songs_with_ids[playlist_songs_with_ids['playlist']==100]
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlist_100)
top_10_user_100 = generate_playlist_recos(songs_filtered, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)

C:\Users\edsta\AppData\Local\Temp\ipykernel_17012\3211393621.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
top_10_user_100

,TrackID,TrackName,Popularity,Duration,Explicit,Artists,ArtistsID,ReleaseYear,Danceability,Energy,...,Tempo,TimeSignature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidates_genre_lists,popularity_red,len,sim
1614,1siUcrRMKOqnrJyknaYrz3,Valder Fields,34,158693,0,['Tamas Wells'],['01osYMBrXUdvSJJlzFgrVi'],2006,0.574,0.385,...,116.986,4,[Tamas Wells],[],[Tamas Wells],Tamas Wells+Valder Fields,[],6,1,0.953569
8687,3zpbWaTHZutGOjT4S4CuUq,The Duck Song,51,191733,0,['Bryant Oden'],['0vuuI2BxGByuCvnBHC00vX'],2009,0.453,0.449,...,184.814,4,[Bryant Oden],[],[Bryant Oden],Bryant Oden+The Duck Song,[],10,1,0.950147
5644,2FXORuBJ8gBDTOVjckYXdF,Remember Me This Way,48,266907,0,['Jordan Hill'],['0OOhKEKPDRy8UrlykRWG2i'],1996,0.471,0.405,...,121.492,4,[Jordan Hill],[],[Jordan Hill],Jordan Hill+Remember Me This Way,[],9,1,0.948939
4875,3Y81Q8dLI74Mgi1wbXvscv,Why Me?,32,204160,0,['Linda Martin'],['1GyPJziXpMB4WOyCPpIDA4'],1992,0.430,0.403,...,152.120,4,[Linda Martin],[],[Linda Martin],Linda Martin+Why Me?,[],6,1,0.945722
6682,1BtwFUWCjt7hfUjN9X5eGX,Nothing'S Gonna Change My Love For You,36,224173,0,['Glenn Medeiros'],['0bByarMN8ryEFQsRo6iCUN'],2013,0.419,0.585,...,142.047,4,[Glenn Medeiros],[],[Glenn Medeiros],Glenn Medeiros+Nothing'S Gonna Change My Love ...,[],7,1,0.944464
2501,1FQ0ecMZMI3ArhcJQPROFP,First Time,52,196493,0,['Robin Beck'],['0tUERnHmr1tKp2pSwPLN8b'],2007,0.581,0.547,...,97.998,4,[Robin Beck],[],[Robin Beck],Robin Beck+First Time,[],10,1,0.943485
9112,0y98eQsCyQ0hyYUsy3GLX7,Lucky,36,251573,0,['Bernie Paul'],['0uI97WlhXD44vyB16hy3PK'],1996,0.629,0.742,...,111.216,4,[Bernie Paul],[],[Bernie Paul],Bernie Paul+Lucky,[],7,1,0.942897
5304,3JQQnhFsJCYAouBbuxVCvJ,Cayman Islands,51,182960,0,['Kings of Convenience'],['41AbNVba2ccpmcc9QtOJE7'],2004,0.413,0.343,...,174.583,4,[Kings of Convenience],[],[Kings of Convenience],Kings of Convenience+Cayman Islands,"[melancholia, norwegian_indie, bergen_indie]",10,1,0.942441
9111,0OjenJcQMRdXI10bQmWYpM,Oh No No,40,244693,0,['Bernie Paul'],['0uI97WlhXD44vyB16hy3PK'],1996,0.706,0.579,...,115.820,4,[Bernie Paul],[],[Bernie Paul],Bernie Paul+Oh No No,[],8,1,0.941703
5296,654YXpi3jNYFjNoqhuYGxw,"Winning A Battle, Losing The War",45,234627,0,['Kings of Convenience'],['41AbNVba2ccpmcc9QtOJE7'],2001,0.587,0.316,...,142.448,4,[Kings of Convenience],[],[Kings of Convenience],"Kings of Convenience+Winning A Battle, Losing ...","[melancholia, norwegian_indie, bergen_indie]",9,1,0.936633


In [48]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

import pandas as pd
import pymssql

app = Dash(__name__)

In [31]:
fig = px.bar(top_40_user_1065, x='TrackName', y='sim')
fig.show()

In [49]:
fig2 = px.bar(top_10_user_100, x='TrackName', y='sim')
fig2.show()

In [51]:
app = Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Dashboard', style={'text-align': 'center'}),


    html.Div([
        html.Label(['Choose a Playlist'],style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='dropdown',
            options=[
                {'label': 'playlist-10697', 'value': '10697'},
                {'label': 'playlist-105', 'value': '105'},
                    ],
            value='10697',
            style={"width": "60%"}),
        
    html.Div(dcc.Graph(id='graph')),        
        ]),

])

@app.callback(
    Output('graph', 'figure'),
    [Input(component_id='dropdown', component_property='value')]
)
def select_graph(value):
    if value == '10697':
        fig = px.bar(top_40_user_1065, x='TrackName', y='sim')
        return fig
    else:
        fig2 = px.bar(top_10_user_100, x='TrackName', y='sim')
        return fig2
    
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Sep/2022 11:52:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2022 11:52:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2022 11:52:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2022 11:52:31] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2022 11:52:31] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2022 11:52:31] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2022 11:52:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2022 11:52:33] "POST /_dash-update-component HTTP/1.1" 200 -
